<a href="https://colab.research.google.com/github/hunkim/ACL-2020-Papers/blob/master/generate_paper_list_with_arxiv_link.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Paper List

In [0]:
def read_papers(path):
    papers = [[]]
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                papers[-1].append(line)
            else:
                papers.append([])
    for p in papers:
        assert len(p) == 2
    return papers

In [48]:
longp = read_papers("./data/long.txt")
longp[:3]

[['2kenize: Tying Subword Sequences for Chinese Script Conversion',
  'Pranav A and Isabelle Augenstein'],
 ['A Batch Normalized Inference Network Keeps the KL Vanishing Away',
  'Qile Zhu, Wei Bi, Xiaojiang Liu, Xiyao Ma, Xiaolin Li and Dapeng Wu'],
 ['A Call for More Rigor in Unsupervised Cross-lingual Learning',
  'Mikel Artetxe, Sebastian Ruder, Dani Yogatama, Gorka Labaka and Eneko Agirre']]

In [49]:
len(longp)

571

In [46]:
short = read_papers("./data/short.txt")
short[:3]

[['A Complete Shift-Reduce Chinese Discourse Parser with Robust Dynamic Oracle',
  'Shyh-Shiun Hung, Hen-Hsen Huang and Hsin-Hsi Chen'],
 ['A Diverse Corpus for Evaluating and Developing English Math Word Problem Solvers',
  'Shen-yun Miao, Chao-Chun Liang and Keh-Yih Su'],
 ['A Frame-based Sentence Representation for Machine Reading Comprehension',
  'Shaoru Guo, Ru Li, Hongye Tan, Xiaoli Li, Yong Guan, Hongyan Zhao and Yueping Zhang']]

In [0]:
len(short)

208

In [0]:
demo = read_papers("./data/demo.txt")
demo[:3]

[['ADVISER: A Toolkit for Developing Multi-modal, Multi-domain and Socially-engaged Conversational Agents',
  'Chia-Yu Li, Daniel Ortega, Dirk Väth, Florian Lux, Lindsey Vanderlyn, Maximilian Schmidt, Michael Neumann, Moritz Völkel, Pavel Denisov, Sabrina Jenne, Zorica Kacarevic and Ngoc Thang Vu'],
 ['BENTO: A Visual Platform for Building Clinical NLP Pipelines Based on CodaLab',
  'Yonghao Jin, Fei Li and Hong Yu'],
 ['Clinical-Coder: Assigning Interpretable ICD-10 Codes to Chinese Clinical Notes',
  'Pengfei Cao, Chenwei Yan, xiangling fu, Yubo Chen, Kang Liu, Jun Zhao, Shengping Liu and Weifeng Chong']]

In [0]:
len(demo)

43

In [9]:
student = read_papers("./data/student.txt")
student[:3]

[['#NotAWhore! A Computational Linguistic Perspective of Rape Culture and Victimization on Social Media',
  'Ashima Suvarna and Grusha Bhalla'],
 ['A Geometry-Inspired Attack for Generating Natural Language Adversarial Examples',
  'Zhao Meng and Roger Wattenhofer'],
 ['A Simple and Effective Dependency parser for Telugu',
  'Sneha Nallani, Manish Shrivastava and Dipti Sharma']]

In [0]:
len(student)

49

# Sorting by Topic



In [18]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk

nltk.download('wordnet')
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
  result=[]
  for token in gensim.utils.simple_preprocess(text) :
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
      result.append(lemmatize_stemming(token))
                
  return result




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#FIXME: Better way to get human readable topic names from LDA topics?
def list2topiclist(list, num_topics = 8):
  processed_docs = []
  for line in list:
    processed_line = preprocess(line[0])
    processed_docs.append(processed_line)

    dictionary = gensim.corpora.Dictionary(processed_docs)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

  
  lda = gensim.models.LdaModel(bow_corpus, num_topics, 
                               id2word = dictionary, passes = 10)


  def get_topic_title(idx, topn=3):
    topn_terms = [dictionary[x[0]] for x in lda.get_topic_terms(idx, topn)]
    return " ".join(topn_terms)

  # Create topic title
  list_topic_titles = []
  for i in range(num_topics):
    list_topic_titles.append(get_topic_title(i))

  # Assign list to topic
  topic_dict = {}
  for line in list:
    processed_line = preprocess(line[0])
    bow_vector = dictionary.doc2bow(processed_line)
    line_topic = sorted(lda.get_document_topics(bow_vector), 
                        key=lambda tup: tup[1], reverse=True)
    topic_title = list_topic_titles[line_topic[0][0]]

    if topic_title not in topic_dict:
      topic_dict[topic_title] = []

    topic_dict[topic_title].append(line)
  
  return topic_dict
      


In [86]:
topic_long = list2topiclist(longp)
for topic in topic_long:
  print(topic)

['model languag dialogu', 'model languag generat', 'languag relat natur', 'machin translat learn', 'cross neural pars', 'generat semant question', 'generat learn model', 'languag word extract']
languag relat natur
generat semant question
cross neural pars
languag word extract
generat learn model
model languag generat
model languag dialogu
machin translat learn


# Search arXiv Link

In [0]:
from googlesearch import search
import urllib
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
from tqdm import tqdm
import time

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


def search_arxiv_link(title, sleep=1):
    time.sleep(sleep)
    link = None
    for j in search(title, tld="co.in", num=10, stop=1, pause=0.5):
        if 'arxiv.org/abs' in j:
            thepage = urllib.request.urlopen(j)
            soup = BeautifulSoup(thepage, "html.parser")
            searched_title = ' '.join(soup.title.text.lower().split()[1:])
            if similarity(title, searched_title) > 0.8:
                link = j
                break
            else:
                print("NOT MATCHED")
                print(title)
                print(searched_title)
    return link

In [0]:
def generate_paper_list_with_arxiv_link(f, papers):
    for p in tqdm(papers):
        title, authors = p
        link = search_arxiv_link(title.lower())
        if link:
            f.write(f"- {title} [[arXiv]]({link})\n")
        else:
            f.write(f"- {title}\n")
    f.write("\n")

In [0]:
def generate_paper_list_with_arxiv_link_topic(f, papers):
  topic_papers = list2topiclist(papers)
  for topic in topic_papers:
    f.write("###" + topic + "\n")
    generate_paper_list_with_arxiv_link(f, topic_papers[topic])


In [0]:
with open("papers_with_arxiv_link_topic.md", "w") as f:
  f.write("## Long Papers\n\n")
  generate_paper_list_with_arxiv_link_topic(f, longp)
  f.write("## Short Papers\n\n")
  generate_paper_list_with_arxiv_link_topic(f, short)
  f.write("## System Demonstrations\n\n")
  generate_paper_list_with_arxiv_link_topic(f, demo)
  f.write("## Student Research Workshop\n\n")
  generate_paper_list_with_arxiv_link_topic(f, student)


['languag model natur', 'graph generat inform', 'learn multi detect', 'semant learn model', 'cross model lingual', 'word generat domain', 'languag natur understand', 'neural translat machin']







  0%|          | 0/87 [00:00<?, ?it/s]




  1%|          | 1/87 [00:02<03:32,  2.48s/it]




  2%|▏         | 2/87 [00:05<03:36,  2.54s/it]




  3%|▎         | 3/87 [00:07<03:33,  2.54s/it]




  5%|▍         | 4/87 [00:09<03:21,  2.43s/it]




  6%|▌         | 5/87 [00:12<03:11,  2.34s/it]




  7%|▋         | 6/87 [00:14<03:09,  2.34s/it]




  8%|▊         | 7/87 [00:16<03:10,  2.38s/it]




  9%|▉         | 8/87 [00:18<03:02,  2.31s/it]

In [0]:
with open("papers_with_arxiv_link.md", "w") as f:
    f.write("## Long Papers\n\n")
    generate_paper_list_with_arxiv_link(f, longp)
    f.write("## Short Papers\n\n")
    generate_paper_list_with_arxiv_link(f, short)
    f.write("## System Demonstrations\n\n")
    generate_paper_list_with_arxiv_link(f, demo)
    f.write("## Student Research Workshop\n\n")
    generate_paper_list_with_arxiv_link(f, student)

  5%|▌         | 31/571 [01:07<21:59,  2.44s/it]

NOT MATCHED
adaptive compression of word embeddings
online embedding compression for text classification using low rank matrix factorization


  6%|▌         | 32/571 [01:10<22:15,  2.48s/it]

NOT MATCHED
addressing posterior collapse with mutual information for improved variational neural machine translation
improved variational neural machine translation by promoting mutual information


  9%|▉         | 53/571 [01:54<18:55,  2.19s/it]

...

  4%|▍         | 2/49 [00:04<01:36,  2.06s/it]

NOT MATCHED
a geometry-inspired attack for generating natural language adversarial examples
a geometry-inspired decision-based attack


 49%|████▉     | 24/49 [00:50<00:53,  2.12s/it]

NOT MATCHED
υbleu: uncertainty-aware automatic evaluation method for open-domain dialogue systems
better automatic evaluation of open-domain dialogue systems with contextualized embeddings


100%|██████████| 49/49 [01:43<00:00,  1.84s/it]
